In [1]:
import socket
import datetime
from dateutil import parser
from _thread import *
from random import randrange, getrandbits
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from base64 import b64encode, b64decode
from math import gcd
import json
import hashlib
import os

In [2]:
def generate_symmetric_key(password):

    password = "mysecret"
    salt = os.urandom(16)  # Generate a random 16-byte salt

    key = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000, dklen=16)
    # Convert bytes to string
    return key

In [3]:
def is_prime(n, k=8):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p

def generate_prime_number(prev, length=8):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 4) or p == prev:
        p = generate_prime_candidate(length)
    return p


def generate_public_key(phi):
    e = 2
    while True:
        if gcd(e, phi) == 1:
            return e
        e += 1

def generate_rsa_key():
    p = generate_prime_number(1,8)
    q = generate_prime_number(p,8)
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)


def encrypt_rsa(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher


def decrypt_rsa(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)


In [4]:
def add_client_info(public_key, key_ring, client_id):
    key_ring[client_id] = public_key
    
def get_client_info(key_ring, client_id):
    return key_ring[client_id]

In [5]:
def add_client_symmetric_key(symmetric_key, key_ring, client_id):
    key_ring[client_id] = symmetric_key

def get_client_symmetric_key(key_ring, client_id):
    return key_ring[client_id]

In [6]:
def encrypt_AES(data, key):
    cipher = AES.new(key, AES.MODE_ECB)
    ct_bytes = cipher.encrypt(pad(data, AES.block_size))
    ct = b64encode(ct_bytes).decode('utf-8')
    return ct

def decrypt_AES(ct, key):
    cipher = AES.new(key, AES.MODE_ECB)
    pt = unpad(cipher.decrypt(b64decode(ct)), AES.block_size)
    return pt

In [7]:
def get_file_hash(file_name):
    try:
        with open(file_name, 'rb') as f:
            data = f.read()
            return hashlib.sha512(data).hexdigest() 
    except:
        return None  

def get_file_digest(file_hash, private_key):
    return encrypt_rsa(private_key, file_hash)

def get_file_base64_str(file_name):
    try:
        with open(file_name, 'rb') as f:
            data = f.read()
            return b64encode(data).decode('utf-8')
    except:
        return None

In [8]:
def get_synchronized_time(clock_server_connection, id):
    #Time at which client sent a request to clockServer
    message = {
        "id": id,
        "message": "SYNC_TIME"
    }
    clock_server_connection.send(str.encode(json.dumps(message)))
    clock_server_request_time = datetime.datetime.now().timestamp()   #T0
    synced_time_str = clock_server_connection.recv(2048).decode()
    clock_data_recv = parser.parse(synced_time_str)
    #Time at which client received a response from the clockServer
    clock_server_response_time = datetime.datetime.now().timestamp() #T1
    latency = clock_server_response_time - clock_server_request_time
    client_time = clock_data_recv + datetime.timedelta(seconds= latency/2)
    return client_time.timestamp()

In [9]:
def handshake_with_KDC(kdc_connection, clock_connection, id, public_key, public_key_ring):
    #send handshake message to KDC
    handshake_message = {
        'id': id,
        'timestamp': get_synchronized_time(clock_connection, id),
        'public_key': public_key
    }
    handshake_message_str = json.dumps(handshake_message)
    kdc_connection.send(str.encode(handshake_message_str))

    #receive handshake response from KDC
    handshake_response_str = kdc_connection.recv(2048).decode('utf-8')
    handshake_response = json.loads(handshake_response_str)
    add_client_info(handshake_response['public_key'], public_key_ring, handshake_response['id'])
    print("Handshake with KDC successful")

In [10]:
database={
    '2019097': 'saatvik',
    '2019096': 'rupanshoo',
    '2019098': 'sam'
}
def verify_password(roll_number, password):
    return database[roll_number] == password

In [11]:
def request_server_public_key_from_KDC(kdc_connection, clock_connection, id, public_key_ring, private_key, requested_id):
    #send public key request for degree granting server to KDC
    message_payload = {
        'requested_id': requested_id
    }
    encrypted_message_payload = encrypt_rsa(private_key, json.dumps(message_payload))
    public_key_request_message = {
        'id': id,
        'timestamp': get_synchronized_time(clock_connection, id),
        'encrypted_payload': encrypted_message_payload,
    }
    kdc_connection.send(str.encode(json.dumps(public_key_request_message)))

    #receive public key for degree granting server from KDC
    public_key_response_str = kdc_connection.recv(2048).decode('utf-8')
    public_key_response = json.loads(public_key_response_str)
    encrypted_payload = public_key_response['encrypted_payload']
    decrypted_payload_str = decrypt_rsa(private_key, encrypted_payload)
    decrypted_payload = json.loads(decrypted_payload_str)
    if(decrypted_payload['requested_id'] != requested_id):
        print('Error: received public key for wrong server')
        return
    else:
        add_client_info(decrypted_payload['requested_public_key'], public_key_ring, decrypted_payload['requested_id'])
    print('Received public key for degree granting server from KDC')

In [12]:
def client_thread(connection, kdc_connection, clock_connection, address, public_key, private_key, id, private_key_ring, public_key_ring):
    while True:
        #receive message from client
        authentication_request = connection.recv(2048).decode('utf-8')
        if not authentication_request:
            break
        authentication_request = json.loads(authentication_request)
        client_id = authentication_request['id']
        encrypted_payload = authentication_request['encrypted_payload']
        decrypted_payload_str = decrypt_rsa(private_key, encrypted_payload)
        decrypted_payload = json.loads(decrypted_payload_str)
        name = decrypted_payload['name']
        password = decrypted_payload['password']
        roll_number = decrypted_payload['roll_number']
        if verify_password(roll_number, password):
            if client_id not in public_key_ring:
                request_server_public_key_from_KDC(kdc_connection, clock_connection, id, public_key_ring, private_key, client_id)         
            symmetric_key = generate_symmetric_key(password)
            add_client_symmetric_key(symmetric_key, private_key_ring, client_id)
            message_payload = {
                'symmetric_key': symmetric_key.hex()
            }
            encrypted_message_payload = encrypt_rsa(get_client_info(public_key_ring,client_id), json.dumps(message_payload))
            response = {
                'id': id,
                'timestamp': get_synchronized_time(clock_connection, id),
                'encrypted_payload': encrypted_message_payload
            }
            connection.send(str.encode(json.dumps(response)))
            while True:
                degree_request = connection.recv(2048).decode('utf-8')
                if not degree_request:
                    break
                degree_request = json.loads(degree_request)
                client_id = degree_request['id']
                encrypted_payload = degree_request['encrypted_payload']
                decrypted_payload_str = decrypt_AES(encrypted_payload, get_client_symmetric_key(private_key_ring, client_id))
                decrypted_payload = json.loads(decrypted_payload_str.decode('utf-8'))
                client_name = decrypted_payload['name']
                client_roll_number = decrypted_payload['roll_number']
                if name == client_name  and roll_number == client_roll_number:
                    print('Client received symmetric key successfully, sending grade card and degree certificate')
                    grade_card_file_digest = get_file_digest(get_file_hash('{}_grade_card.txt'.format(client_roll_number)), private_key)
                    degree_certificate_file_digest = get_file_digest(get_file_hash('{}_degree_certificate.txt'.format(client_roll_number)), private_key)
                    file_response_payload = {
                        'grade_card_file_digest': grade_card_file_digest,
                        'degree_certificate_file_digest': degree_certificate_file_digest,
                        'grade_card_file': get_file_base64_str('{}_grade_card.txt'.format(client_roll_number)),
                        'degree_certificate_file': get_file_base64_str('{}_degree_certificate.txt'.format(client_roll_number))
                    }
                    encrypted_file_response_payload = encrypt_AES(str.encode(json.dumps(file_response_payload)), get_client_symmetric_key(private_key_ring, client_id))
                    file_response = {
                        'id': id,
                        'timestamp': get_synchronized_time(clock_connection, id),
                        'encrypted_payload': encrypted_file_response_payload
                    }
                    connection.send(str.encode(json.dumps(file_response)))
                    return
        else:
            response = {
                'id': id,
                'timestamp': get_synchronized_time(clock_connection, id),
                'message': 'Authentication failed'
            }
            connection.send(str.encode(json.dumps(response)))

In [13]:
host = '127.0.0.1'
clock_port = 8080
certificate_port = 5000
kdc_port = 6666
id='degree-granting-server'
public_key, private_key = generate_rsa_key()
public_key_ring = {}
private_key_ring = {}
certificate_server_socket = socket.socket()
certificate_server_socket.bind((host, certificate_port))

clock_server_connection = socket.socket()
clock_server_connection.connect((host, clock_port))

kdc_connection = socket.socket()
kdc_connection.connect((host, kdc_port))



print("Certificate granting server is accepting connections now...")
certificate_server_socket.listen(5)

handshake_with_KDC(kdc_connection, clock_server_connection, id, public_key, public_key_ring)

while True:
	client, client_address = certificate_server_socket.accept()
	print('Connected to: ' + client_address[0] + ':' + str(client_address[1]))
	start_new_thread(client_thread, (client, kdc_connection, clock_server_connection, client_address, public_key, private_key, id, private_key_ring, public_key_ring))



Certificate granting server is accepting connections now...
Handshake with KDC successful
Connected to: 127.0.0.1:63011
Received public key for degree granting server from KDC
Client received symmetric key successfully, sending grade card and degree certificate
